In [1]:
LOAD_MODEL = 'models/attention2.01.pt'
MODEL_CHECKPOINT = 'models/attention2.01.pt'
DATASET_PATH = 'data/interim/preprocessed_paranmt3.tsv'

In [2]:
import torch
import numpy as np
from torch.utils.data import DataLoader

import os
os.chdir("..") # go to the root dir

# Get the Dataset

In [3]:
MAX_SENT_SIZE = 32
MAX_TOKENS = 30_000

In [4]:
from src.data.make_dataset import ParanmtDataset

train_dataset = ParanmtDataset(
    path=DATASET_PATH,
    max_sent_size=MAX_SENT_SIZE,
    train=True,
    seed=42,
    take_first=400_000,
)

In [5]:
train_dataset.build_vocab(
    min_freq=2,
    specials=['<unk>', '<pad>', '<sos>', '<eos>'],
    max_tokens=MAX_TOKENS,
)

In [6]:
enc_vocab = train_dataset.toxic_vocab
dec_vocab = train_dataset.neutral_vocab

print("size of encoder vocab:", len(enc_vocab))
print("size of decoder vocab:", len(dec_vocab))

size of encoder vocab: 30000
size of decoder vocab: 30000


In [7]:
val_dataset = ParanmtDataset(
    path=DATASET_PATH,
    max_sent_size=MAX_SENT_SIZE,
    vocabs=(enc_vocab, dec_vocab), # avoid data leakage
    train=False,
    seed=42,
    take_first=10_000,
)

In [8]:
train_dataset.df

,similarity,lenght_diff,toxic_sent,neutral_sent,toxic_val,neutral_val
0,0.834750,0.390805,"[do, you, know, that, you, flirt, with, every,...","[you, flirt, with, every, woman, you, come, in...",0.998895,0.000226
1,0.769013,0.102041,"[yes, ,, but, she, thinks, you, are, a, raving...","[yes, ,, but, she, thought, you, were, a, stre...",0.957565,0.046821
2,0.879764,0.064516,"[but, they, will, not, do, shit, about, them, ...","[but, they, do, not, do, anything, about, it, ...",0.998256,0.018739
3,0.689452,0.153846,"[you, would, die, .]","[and, you, die, ?]",0.995866,0.070247
4,0.765473,0.206897,"[same, as, piss, or, eating, you, are, like, a...","[it, is, like, peeing, ,, eating, you, are, ac...",0.991141,0.078439
...,...,...,...,...,...,...
399995,0.726744,0.211538,"[and, i, suggest, you, take, your, 4, thousand...","[and, i, suggest, you, take, your, $, 4, 000, ...",0.996931,0.001203
399996,0.760384,0.173913,"[a, mass, death, sentence, for, krakovich, ,, ...","[that, was, the, joint, death, certificate, of...",0.941657,0.000042
399997,0.779441,0.085366,"[someone, is, trying, to, kill, me, ,, sending...","[someone, is, trying, to, kill, me, and, he, i...",0.836793,0.024553
399998,0.800366,0.213115,"[if, you, are, here, for, soup, ,, i, will, br...","[if, you, are, here, for, the, gumbo, ,, i, am...",0.645919,0.061810


In [9]:
len(train_dataset), len(val_dataset)

(400000, 10000)

# Build the Dataloaders

In [10]:
batch_size = 256

In [11]:
train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
)

In [12]:
# let's check if shape and everything is ok
for batch in train_dataloader:
    toxic_sent, neutral_sent = batch
    print("toxic_sent.shape:", toxic_sent.shape)
    print("neutral_sent.shape:", neutral_sent.shape)
    break

toxic_sent.shape: torch.Size([256, 32])
neutral_sent.shape: torch.Size([256, 32])


In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# Load the Model

- EncoderDecoder (Seq2Seq) with Attention mechanism

In [14]:
from src.models.attention.encoder import Encoder
from src.models.attention.decoder import Decoder
from src.models.attention.attention import Attention
from src.models.attention import Seq2SeqAttention

In [15]:
## Encoder
enc_input_dim = len(enc_vocab)
enc_embed_dim = 128
enc_hidden_dim = 256
enc_dropout = 0.1
enc_padding_idx = enc_vocab['<pad>']

## Decoder
dec_output_dim = len(dec_vocab)
dec_embed_dim = 128
dec_hidden_dim = 256
dec_dropout = 0.1
dec_padding_idx = dec_vocab['<pad>']

In [16]:
# load the encoder and decoder for our model
encoder = Encoder(
    input_dim=enc_input_dim,
    embed_dim=enc_embed_dim,
    hidden_dim=enc_hidden_dim,
    dec_hidden_dim=dec_hidden_dim,
    dropout=enc_dropout,
    vocab=enc_vocab,
    padding_idx=enc_padding_idx,
).to(device)

attention = Attention(
    enc_hidden_dim,
    dec_hidden_dim,
)

decoder = Decoder(
    output_dim=dec_output_dim,
    embed_dim=dec_embed_dim,
    hidden_dim=dec_hidden_dim,
    attention=attention,
    enc_hidden_dim=enc_hidden_dim,
    dropout=dec_dropout,
    vocab=dec_vocab,
    padding_idx=dec_padding_idx,
).to(device)

In [17]:
best_loss = float('inf')

model = Seq2SeqAttention(
    encoder=encoder,
    decoder=decoder,
    device=device,
    max_sent_size=MAX_SENT_SIZE,
).to(device)

In [18]:
from src.models.utils import count_parameters

print(f"number of parameters in model: {count_parameters(model)//1e6}M")

number of parameters in model: 142.0M


In [19]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
criterion = torch.nn.CrossEntropyLoss(ignore_index=dec_vocab['<pad>'])

In [20]:
from src.models.train_model import train

best_loss = train(
    model=model,
    loaders=(train_dataloader, val_dataloader),
    optimizer=optimizer,
    criterion=criterion,
    epochs=10,
    device=device,
    best_loss=best_loss,
    ckpt_path=MODEL_CHECKPOINT,
    clip_grad=1,
    teacher_force={
        'value': 1,
        'gamma': 0.8,
        'update_every_n_epoch': 1,
    } # first 10 epoch teacher force 1, after it will be turned off
)

Evaluating 2: 100%|██████████| 40/40 [00:13<00:00,  2.88it/s, loss=5.07]


Update teacher force to 0.6400000000000001


Evaluating 3: 100%|██████████| 40/40 [00:13<00:00,  2.89it/s, loss=4.54]


Update teacher force to 0.5120000000000001


Evaluating 4: 100%|██████████| 40/40 [00:14<00:00,  2.72it/s, loss=4.35]


...

Update teacher force to 0.1677721600000001


Evaluating 9: 100%|██████████| 40/40 [00:13<00:00,  2.88it/s, loss=4.03]


Update teacher force to 0.13421772800000006


Evaluating 10: 100%|██████████| 40/40 [00:13<00:00,  2.91it/s, loss=3.99]


Update teacher force to 0.10737418240000006


In [21]:
# let's load the model and predict
model = torch.load(MODEL_CHECKPOINT)
model.to(device)
model.eval()

Seq2SeqAttention(
  (encoder): Encoder(
    (vocab): Vocab()
    (embedding): Embedding(30000, 128, padding_idx=1)
    (rnn): GRU(128, 256, batch_first=True, bidirectional=True)
    (fc): Linear(in_features=512, out_features=256, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (decoder): Decoder(
    (vocab): Vocab()
    (embedding): Embedding(30000, 128, padding_idx=1)
    (rnn): GRU(640, 256, batch_first=True)
    (attention): Attention(
      (attn): Linear(in_features=768, out_features=256, bias=True)
      (v): Linear(in_features=256, out_features=1, bias=False)
    )
    (dropout): Dropout(p=0.1, inplace=False)
    (fc_out): Sequential(
      (0): Linear(in_features=896, out_features=2048, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.1, inplace=False)
      (3): Linear(in_features=2048, out_features=4096, bias=True)
      (4): ReLU()
      (5): Dropout(p=0.1, inplace=False)
      (6): Linear(in_features=4096, out_features=30000, bias=True)
    )
  )
)

In [23]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
detokenizer = TreebankWordDetokenizer()

# let's see how our model works
num_examples = 10
num_sentence = 3
dataset = train_dataset
for _ in range(num_examples):
    idx = np.random.randint(0, len(dataset))
    toxic_sent = detokenizer.detokenize(dataset.df.loc[idx, 'toxic_sent'])
    neutral_sent = detokenizer.detokenize(dataset.df.loc[idx, 'neutral_sent'])
    
    print('toxic_sent:', toxic_sent)
    print('neutral_sent:', neutral_sent)
    preds = model.predict(
        toxic_sent,
        beam=True,
        beam_search_num_candidates=num_sentence,
        post_process_text=False,
        use_encoder_out=True,
    ) # let's use beam search
    print("predictions:")
    for i in range(num_sentence):
        print(f"\t{i+1})", preds[i])
    print("\n")

toxic_sent: sarah i dead, and it is your fault.
neutral_sent: sarah i death is your fault.
predictions:
	1) ['sarah', 'i', 'dead', ',', 'and', 'it', 'is', 'your', 'fault', '.', '<eos>']
	2) ['sarah', 'i', 'dead', ',', 'and', 'it', 'is', 'your', 'fault', '.', '.', '<eos>']
	3) ['sarah', 'i', 'dead', ',', 'it', 'is', 'your', 'fault', '.', '<eos>']


toxic_sent: my last girlfriend, the feminist vegetarian whore broke up with me, saying i was too excited.
neutral_sent: my last girlfriend, a feminist vegan punk, broke up with me because . i was too angry.
predictions:
	1) ['my', 'last', 'girlfriend', ',', 'the', 'feminist', 'vegan', 'vegan', 'was', 'broke', 'up', 'with', 'me', ',']
	2) ['my', 'last', 'girl', ',', 'the', 'feminist', 'vegan', 'vegan', 'broke', 'me', 'with', 'me', 'said', 'i']
	3) ['my', 'last', 'girl', ',', ',', 'feminist', 'vegan', 'vegan', 'was', 'broke', 'with']


toxic_sent: you know what kind of bitches we girls can be, rosie said, laughing.
neutral_sent: you know how bi